In [1]:
import os
import time
import pandas as pd
import numpy as np
import pathlib
from io import StringIO
from Bio import SeqIO
from Bio.Cluster import distancematrix
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import subprocess
from matplotlib import pyplot as plt

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

MagicBlast:

- raw reads to nt
- contigs to nt
- raw reads to gsa_virus
- contigs to gsa_virus

In [3]:
PRJ='PRJNA605983'
DATA_PATH=f'/mnt/1TB_0/Data/Assembly/{PRJ}/'
DB='bsl_complete_nuclotide'

In [4]:
def clean_string(s):
    s = s.replace(',', '').strip()
    return s

In [5]:
def get_contigs_ascessions(contigs_file):
    contigs=[]
    accessions=[]
    with open(contigs_file, 'r') as temp_f:
        for line in temp_f:
            if len(line.strip())>0:
                vals=line.split("\t")
                contigs.append(clean_string(vals[0]))
                accessions.append(clean_string(vals[2]))
    return  contigs, accessions


In [6]:
def get_raw_sam_ascessions(reads_file, machine_id='v300043428'):
    accessions=[]
    with open(reads_file, 'r') as temp_f:
        for line in temp_f:
            if len(line.strip())>0:
                if line.startswith(machine_id) or line.startswith(f'@{machine_id}'):
                    vals=line.split("\t")
                    accessions.append(clean_string(vals[2]))
    return accessions

In [7]:
def get_val_count(accessions):
    values, counts = np.unique(accessions, return_counts=True)
    idx = np.argsort(counts)[::-1]
    values = np.array(values)[idx]
    counts = np.array(counts)[idx]
    return values, counts, idx

In [8]:
def get_titles(df, values):
    titles=[]
    for v in values:
        l=df.loc[df.accession==v, 'title'].tolist()
        titles.append(l[0])
    return titles

In [9]:
def get_accession_dat(values, dbname='nt'):
    vdats=[]
    for v in values:
        try:
            vdat=!blastdbcmd -db $dbname -entry $v
        except Exception as e:
            if dbname!='nt':
                vdat=!blastdbcmd -db 'nt' -entry $v
        if vdat is not None:
            vdats.append(vdat[0].split(',')[0])
    return vdats

### List of accessions being searched for

In [43]:
all_accessions=!blastdbcmd -db bsl_complete_nuclotide -entry all

In [51]:
accession_titles=[a for a in all_accessions if a.startswith('>')]

In [52]:
accession_titles

['>2OJ3_B Chain B, Hdv Ribozyme',
 ">AC105600.5 Rattus norvegicus 4 BAC CH230-209E1 (Children's Hospital Oakland Research Institute) complete sequence",
 '>AC123074.2 Mus musculus BAC clone RP23-168H19 from 10, complete sequence',
 '>ACVO01000021.1 Rothia mucilaginosa ATCC 25296 contig00003, whole genome shotgun sequence',
 '>AF274751.1 Bacteriophage S13, complete genome',
 '>AJ289709.1 Human endogenous retrovirus H HERV-H/env62 proviral copy, clone 231E12',
 '>NC_000932.1 Arabidopsis thaliana chloroplast, complete genome',
 '>AY029768.1 Nipah virus isolate UMMC2, complete genome',
 '>AY988601.1 Nipah virus from Bangladesh, complete genome',
 '>DJ040595.1 A METHOD OF DETECTING NIPAH VIRUS AND METHOD FOR PROVIDING IMMUNOPROTECTION AGAINST HENIPAVIRUSES',
 '>DQ011855.1 Porcine hemagglutinating encephalomyelitis virus strain VW572, complete genome',
 '>DQ022305.2 Bat SARS coronavirus HKU3-1, complete genome',
 '>DQ079883.1 Coliphage ID45, complete genome',
 '>DQ079895.1 Coliphage WA11, co

## Raw Reads

In [12]:
#sra_list= ['SRR11092059','SRR11092060']

In [13]:
sra_list= ['SRR11092059','SRR11092060','SRR11092061','SRR11092062','SRR11092063','SRR11092064','SRR11092056','SRR11092057','SRR11092058']

In [15]:
for sra in sra_list:
    out_path=f'{DATA_PATH}{sra}/magic_blast/'
    sam_out=f'{out_path}{sra}_magicBLAST_{DB}.sam'
    machine_id='v300043428'
    if sra in ['SRR11092056','SRR11092057','SRR11092058', 'SRR11092064']:
        machine_id='M04943'
    accessions=get_raw_sam_ascessions(sam_out, machine_id=machine_id)
    values, counts, idx = get_val_count(accessions)
    titles=get_accession_dat(values, DB)
    print(f'\n{sra}')
    for t, v, c in zip(titles, values, counts):
        print(f'{t}, {v}, {c}')


SRR11092059
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 4778613
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 1539112
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 920132
>NZ_JRYB01000001.1 Massilia timonae strain NEU LO55.unitig, NZ_JRYB01000001.1, 624772
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 371133
>7 CFTR [organism=Homo sapiens] [GeneID=1080] [chromosome=7], 7, 312814
>2 ABCG2 [organism=Homo sapiens] [GeneID=9429] [chromosome=4], 2, 207423
>MT241668.1 Leopoldamys sabanus voucher MZF1958 mitochondrion, MT241668.1, 47485
>MK433563.1 Cloning vector pVAX1-BMP2, MK433563.1, 46114
>9 BIRC5 [organism=Homo sapiens] [GeneID=332] [chromosome=17], 9, 28734
>MN996867.1 Cloning vector pcDNA3.1_+, MN996867.1, 18220
>KY199425.1 Synthetic construct H7N9 HA gene, KY199425.1, 16382
>NC_026425.1 Influenza A virus (A/Shanghai/02/2013(H7N9)) segment 4 hemagglutinin (HA) gene, NC_026425.1, 15830
>6 CAV1 [organism=Homo sapiens


SRR11092061
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 12920740
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 2142585
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 1294917
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 1131494
>2 ABCG2 [organism=Homo sapiens] [GeneID=9429] [chromosome=4], 2, 985480
>7 CFTR [organism=Homo sapiens] [GeneID=1080] [chromosome=7], 7, 508785
>9 BIRC5 [organism=Homo sapiens] [GeneID=332] [chromosome=17], 9, 180551
>MT241668.1 Leopoldamys sabanus voucher MZF1958 mitochondrion, MT241668.1, 153743
>6 CAV1 [organism=Homo sapiens] [GeneID=857] [chromosome=7], 6, 98486
>1 RPS6KA1 [organism=Homo sapiens] [GeneID=6195] [chromosome=1], 1, 82001
>10 TP53 [organism=Homo sapiens] [GeneID=7157] [chromosome=17], 10, 55472
>NZ_JRYB01000001.1 Massilia timonae strain NEU LO55.unitig, NZ_JRYB01000001.1, 53716
>3 FGF2 [organism=Homo sapiens] [GeneID=2247] [chromosome=4], 3, 45825
>4 TERT [organism=Homo sapi


SRR11092063
>2 ABCG2 [organism=Homo sapiens] [GeneID=9429] [chromosome=4], 2, 1477321
>7 CFTR [organism=Homo sapiens] [GeneID=1080] [chromosome=7], 7, 857756
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 685658
>9 BIRC5 [organism=Homo sapiens] [GeneID=332] [chromosome=17], 9, 241416
>1 RPS6KA1 [organism=Homo sapiens] [GeneID=6195] [chromosome=1], 1, 130982
>6 CAV1 [organism=Homo sapiens] [GeneID=857] [chromosome=7], 6, 128548
>10 TP53 [organism=Homo sapiens] [GeneID=7157] [chromosome=17], 10, 90864
>3 FGF2 [organism=Homo sapiens] [GeneID=2247] [chromosome=4], 3, 65892
>AJ289709.1 Human endogenous retrovirus H HERV-H/env62 proviral copy, AJ289709.1, 55572
>4 TERT [organism=Homo sapiens] [GeneID=7015] [chromosome=5], 4, 51739
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 44323
>11 BAX [organism=Homo sapiens] [GeneID=581] [chromosome=19], 11, 36704
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 29968
>12 SOD1 [organism=Homo sapiens] [G


SRR11092057
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 406875
>7 CFTR [organism=Homo sapiens] [GeneID=1080] [chromosome=7], 7, 179056
>2 ABCG2 [organism=Homo sapiens] [GeneID=9429] [chromosome=4], 2, 114508
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 53483
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 37872
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 31131
>9 BIRC5 [organism=Homo sapiens] [GeneID=332] [chromosome=17], 9, 13575
>AJ289709.1 Human endogenous retrovirus H HERV-H/env62 proviral copy, AJ289709.1, 13505
>4 TERT [organism=Homo sapiens] [GeneID=7015] [chromosome=5], 4, 13078
>1 RPS6KA1 [organism=Homo sapiens] [GeneID=6195] [chromosome=1], 1, 10779
>3 FGF2 [organism=Homo sapiens] [GeneID=2247] [chromosome=4], 3, 9531
>6 CAV1 [organism=Homo sapiens] [GeneID=857] [chromosome=7], 6, 8158
>AF274751.1 Bacteriophage S13, AF274751.1, 6820
>10 TP53 [organism=Homo sapiens] [GeneID=7157] [chromosome=17], 10, 6531

### Lane 02 reads

In [18]:
sra_list= ['SRR11092062','SRR11092063']

In [19]:
for sra in sra_list:
    out_path=f'{DATA_PATH}{sra}/magic_blast/'
    sam_out=f'{out_path}{sra}_L02_magicBLAST_{DB}.sam'
    machine_id='v300043428'
    if sra in ['SRR11092056','SRR11092057','SRR11092058', 'SRR11092064']:
        machine_id='M04943'
    accessions=get_raw_sam_ascessions(sam_out, machine_id=machine_id)
    values, counts, idx = get_val_count(accessions)
    titles=get_accession_dat(values, DB)
    print(f'\n{sra}')
    for t, v, c in zip(titles, values, counts):
        print(f'{t}, {v}, {c}')


SRR11092062
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 1241984
>2 ABCG2 [organism=Homo sapiens] [GeneID=9429] [chromosome=4], 2, 1037656
>7 CFTR [organism=Homo sapiens] [GeneID=1080] [chromosome=7], 7, 641744
>9 BIRC5 [organism=Homo sapiens] [GeneID=332] [chromosome=17], 9, 178401
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 132958
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 105692
>6 CAV1 [organism=Homo sapiens] [GeneID=857] [chromosome=7], 6, 96903
>1 RPS6KA1 [organism=Homo sapiens] [GeneID=6195] [chromosome=1], 1, 81729
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 70224
>10 TP53 [organism=Homo sapiens] [GeneID=7157] [chromosome=17], 10, 56338
>3 FGF2 [organism=Homo sapiens] [GeneID=2247] [chromosome=4], 3, 48022
>4 TERT [organism=Homo sapiens] [GeneID=7015] [chromosome=5], 4, 40932
>AJ289709.1 Human endogenous retrovirus H HERV-H/env62 proviral copy, AJ289709.1, 38586
>11 BAX [organism=Homo sapiens] [GeneID